In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, GRU, Dense
from transformers import AutoTokenizer
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

2024-05-19 16:20:16.789920: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 16:20:16.790059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 16:20:17.057452: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
# Sample data preparation (assuming you have your data in a pandas DataFrame)
# Replace this with your actual DataFrame
data = pd.DataFrame({
    'input_text': ['i ike eting cakes', 'i go to shool everydy'],
    'target_text': ['i like eating cakes', 'i go to school everyday']
})

data = pd.read_csv('/kaggle/input/dop-test-files/errors.csv').drop(columns=['Unnamed: 0'])
data

,label,preds
0,иностранный агент,иностранный аген
1,свидетельствуют о проблемах с печенью,свидетельствуют о праблемах спецнью
2,найдите способ быть полезными другим людям,найдиче сьпособ быть полезном другим людем
3,я уже поставил белье в стирку,я уже поставил бельо встиру
4,круглый мяч,круглый мядчь
...,...,...
1475,летим в отпуск на гавайи,влетим в отпуск наговаи
1476,кротовая настойка,кротовая на стойка
1477,думаю нам пора расходиться по домам,домаю ном порарасходится по домам
1478,красивые цветы украшают сад,красивые цветы укрошают сад


In [31]:
# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['preds'].tolist() + data['label'].tolist())

# Convert text to sequences
input_sequences = tokenizer.texts_to_sequences(data['preds'].tolist())
target_sequences = tokenizer.texts_to_sequences(data['label'].tolist())

# Padding sequences to the same length
max_seq_len = max(max(len(seq) for seq in input_sequences), max(len(seq) for seq in target_sequences))
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='post')
padded_target_sequences = pad_sequences(target_sequences, maxlen=max_seq_len, padding='post')

# Define model parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 1024
rnn_units = 512

In [41]:
input_sequences[:10]

[[585, 1308],
 [586, 26, 1309, 1310],
 [1311, 1312, 30, 1313, 262, 1314],
 [4, 24, 381, 1315, 1316],
 [587, 1317],
 [263, 32, 1318],
 [588, 1319, 1320, 264],
 [589, 1321],
 [1322, 590, 2, 1323, 1324, 18, 203],
 [164, 204, 2, 265, 1325]]

In [42]:
target_sequences[:10]

[[585, 4166],
 [586, 26, 4167, 7, 4168],
 [4169, 160, 30, 4170, 262, 4171],
 [4, 24, 381, 1166, 1, 4172],
 [587, 4173],
 [263, 32, 4174],
 [588, 551, 4175, 264],
 [589, 4176],
 [4177, 590, 2, 4178, 131, 18, 203],
 [164, 204, 2, 265, 165]]

In [32]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_len))
model.add(LSTM(rnn_units, return_sequences=True))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Prepare target data for training
padded_target_sequences = np.expand_dims(padded_target_sequences, -1)

# Train the model
model.fit(padded_input_sequences, padded_target_sequences, epochs=50, batch_size=50)

Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.6166 - loss: 5.2630
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7100 - loss: 2.6536
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.7089 - loss: 2.5459
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.7055 - loss: 2.4733
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7180 - loss: 2.2910
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7084 - loss: 2.2849
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7151 - loss: 2.1496
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7204 - loss: 2.0189
Epoch 9/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7166 - loss: 1.9433
Epoch 10/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7232 - loss: 1.8184
Epoch 11/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7303 - loss: 1.7006
Epoch 12/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy:

In [39]:
def predict(input_text):
    # Preprocess the input text
    input_seq = tokenizer.texts_to_sequences([input_text])
    padded_input_seq = pad_sequences(input_seq, maxlen=max_seq_len, padding='post')

    # Predict the output sequence
    predictions = model.predict(padded_input_seq)
    predicted_sequence = np.argmax(predictions, axis=-1)
    
    # Convert the predicted sequence back to text
    decoded_sentence = ' '.join(tokenizer.index_word.get(index, '') for index in predicted_sequence[0])
    
    return decoded_sentence.strip()

# Example prediction
corrected_text = predict('мотивация должна быыыть всегдаааа')
print('Corrected Text:', corrected_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Corrected Text: 


FINE TUNING OF PRETRAINED CORRECTOR

In [12]:
!pip install datasets
!apt install git-lfs
!pip install transformers
!pip install sentencepiece 
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0515edb524f206b0d599dc648676e7419cf901269d018c935b3b02118ca48c00
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [11]:
# Импорт библиотек
import numpy as np
from datasets import Dataset
import tensorflow as tf
import nltk
from transformers import T5TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from transformers.optimization import Adafactor, AdafactorSchedule
from datasets import load_dataset, load_metric

SyntaxError: invalid syntax (2283669381.py, line 4)

In [ ]:
# загрузка параметров
raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")
nltk.download('punkt')

In [10]:
# Определение параметров
#REPO = "t5-russian-spell"  # Введите наазвание название репозитория
MODEL_NAME = "UrukHan/t5-russian-spell" # Введите наазвание выбранной модели из хаба
MAX_INPUT = 256  # Введите максимальную длинну входных данных  в токенах (длинна входных фраз в словах (можно считать полслова токен))
MAX_OUTPUT  = 256 # Введите максимальную длинну прогнозов в токенах (можно уменьшить для задач суммризации или других задач где выход короче)
BATCH_SIZE = 8 
df = pd.read_csv('/kaggle/input/dop-test-files/errors.csv').drop(columns=['Unnamed: 0'])
df.head(3) # Введите наазвание название датасета

,label,preds
0,иностранный агент,иностранный аген
1,свидетельствуют о проблемах с печенью,свидетельствуют о праблемах спецнью
2,найдите способ быть полезными другим людям,найдиче сьпособ быть полезном другим людем


In [ ]:
data = Dataset.from_pandas(df)

In [ ]:
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [ ]:
model.config.max_length = MAX_OUTPUT

In [ ]:
train = data['train']
test = data['test'].train_test_split(0.02)['test']